# Exploratory Data Analysis on Job Reviews

In [3]:
import pandas as pd
import sqlite3
import cufflinks as cf
from textblob import TextBlob
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Reading the data

In [6]:
con = sqlite3.connect("collectors/data.sqlite3")
df = pd.read_sql_query("SELECT * from company_review", con)
con.close()

In [7]:
df.head(10)

,id,company,title,rating,author,author_status,location,date,description,source
0,1,bmo financial group,Decent,3.0,Credit Analyst (Current Employee),N/A,"Burnaby, BC",11 July 2018,Not a bad job but can get boring easily. Manag...,indeed.com
1,2,bmo financial group,easy place to work,3.0,Project Manager (Former Employee),N/A,"Toronto, ON",7 April 2020,co-workers are friendly but pay is not good at...,indeed.com
2,3,bmo financial group,Dead Industry- They will squeeze you like a le...,1.0,Assistant Manager (Former Employee),N/A,"Cambridge, ON",6 April 2020,They give you the illusion of how great of a j...,indeed.com
3,4,bmo financial group,"In BMO agile team, one developer has to servic...",2.0,Software Specialist (Former Employee),N/A,"Toronto, ON",6 April 2020,"Basically, in BMO agile team, they hire 5+ per...",indeed.com
4,5,bmo financial group,Good,5.0,Relationship Manager (Current Employee),N/A,"Nanaimo, BC",4 April 2020,Work/life balance and customer centric. Traini...,indeed.com
5,6,bmo financial group,Inclusive and Diverse,5.0,Customer customer representative (Current Empl...,N/A,"Maple Ridge, BC",4 April 2020,A typical work day includes problem solving cu...,indeed.com
6,7,bmo financial group,Understanding workplace with flexibility and p...,5.0,Customer Service Representative (Current Emplo...,N/A,"Micmac Village, NS",2 April 2020,I am currently a student working part time at ...,indeed.com
7,8,bmo financial group,bmo is the best company I have worked for,5.0,Victoria- Customer Service Representative (Cur...,N/A,"Victoria, BC",1 April 2020,everyone I so supportive and will work with yo...,indeed.com
8,9,bmo financial group,Great place to build a career and friends!,4.0,"Director, Payment Cards Services (Former Emplo...",N/A,"Toronto, ON",1 April 2020,The best part of working at a company like BMO...,indeed.com
9,10,bmo financial group,Outstanding leadership support and opportuniti...,5.0,Customer Contact Centre Representative (Curren...,N/A,"Mississauga, ON",31 March 2020,BMO's culture is nothing like i've seen before...,indeed.com


# Data Preprocessing

In [8]:
df = df[~df['description'].isnull()]

def preprocess(DescriptionText):
    DescriptionText = DescriptionText.str.replace("(<br/>)", "")
    DescriptionText = DescriptionText.str.replace('(<a).*(>).*(</a>)', '')
    DescriptionText = DescriptionText.str.replace('(&amp)', '')
    DescriptionText = DescriptionText.str.replace('(&gt)', '')
    DescriptionText = DescriptionText.str.replace('(&lt)', '')
    DescriptionText = DescriptionText.str.replace('(\xa0)', ' ')  
    return DescriptionText
df['description'] = preprocess(df['description'])

In [9]:
df['review_len'] = df['description'].astype(str).apply(len)
df['word_count'] = df['description'].apply(lambda x: len(str(x).split()))

In [10]:
#Distribution of Review Ratings
df['rating'].iplot(
    kind='hist',
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    color ='Purple',
    title='Review Rating Distribution')

In [11]:
#The distribution review text lengths
df['review_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    color ='Cyan',
    title='Review Text Length Distribution')

In [12]:
# The distribution of review word count
df['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    color = 'Grey',
    title='Review Text Word Count Distribution')

In [13]:
# top 20 words in description before removing stop keyword
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=None):
 
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



common_words = get_top_n_words(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Red', title='Top 20 words in review-description before removing stop words')

In [14]:
# top 20 words in description after removing stop keyword
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Green', title='Top 20 words in job-review after removing stop words')

In [15]:
#The distribution of top bigrams before removing stop words
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df3 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df3.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Red', title='Top 20 bigrams in the job-review before removing stop words')

In [16]:
#The distribution of top bigrams after removing stop words
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='Green',title='Top 20 bigrams in the job-review after removing stop words')

In [17]:
#The distribution of Top trigrams before removing stop words

def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df5 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df5.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='Red',title='Top 20 trigrams in the job-review before removing stop words')


In [18]:
#The distribution of Top trigrams after removing stop words

def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df6 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df6.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Green', title='Top 20 trigrams in the job-review after removing stop words')